## Define Libraries

In [ ]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

### Function

#### expand_calc_features

In [ ]:
import pandas as pd
import json

def expand_calc_features(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_list = []

    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
            calc_features_list.append(features_dict)
        except (json.JSONDecodeError, AttributeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            calc_features_list.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_list)

    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')

    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)

    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    return result_df


#### expand_calc_features_robust

In [ ]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns with better error handling.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_data = []

    for idx, row in df.iterrows():
        calc_features_str = row['calcFeatures']

        if pd.isna(calc_features_str) or calc_features_str == '':
            calc_features_data.append({})
            continue

        try:
            # Clean the string and parse JSON
            cleaned_str = calc_features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            features_dict = json.loads(cleaned_str)
            calc_features_data.append(features_dict)
        except Exception as e:
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"Problematic string: {calc_features_str[:100]}...")  # Print first 100 chars
            calc_features_data.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)

    # Add prefix to JSON-derived columns to avoid conflicts with existing columns
    calc_features_df = calc_features_df.add_prefix('feat_')

    # Combine DataFrames
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    print(f"Original DataFrame shape: {df.shape}")
    print(f"Expanded DataFrame shape: {result_df.shape}")
    print(f"Added {len(calc_features_df.columns)} new columns from calcFeatures")

    return result_df

#### transform_data

In [ ]:
import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List

def transform_data(
    d1: pd.DataFrame, 
    feature_column: List[str], 
    a: str = 'demo_score', 
    modelDisplayName: str = 'Cash_beta_trench1_Demo_backscore', 
    tc: str = "", 
    subscription_name: str = 'sil_march 25 models'
) -> pd.DataFrame:
    """
    Transforms input data into a structured format suitable for model scoring output.

    Parameters:
    - d1 (pd.DataFrame): Input DataFrame containing raw data.
    - feature_column (List[str]): List of column names to include in the 'calcFeature' JSON.
    - a (str): Column name containing the prediction score. Default is 'demo_score'.
    - modelDisplayName (str): Name of the model used for scoring.
    - tc (str): Trench category (optional).
    - do (str): Device operating system. Default is 'android'.
    - subscription_name (str): Name of the subscription or model group.

    Returns:
    - pd.DataFrame: Transformed DataFrame with structured output.
    """

    # Make a copy of the input DataFrame to avoid modifying the original
    df = d1.copy()
    
    # Initialize an empty list to store transformed rows
    output_data = []
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Initialize dictionary to hold feature values
        calc_feature = {}
        
        # Loop through each feature column and extract its value from the row
        for col in feature_column:
            if col in row and pd.notna(row[col]):
                # Convert datetime values to ISO format strings
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
        # Get the current timestamp for start_time, end_time, and publish_time
        current_time = datetime.now().isoformat()
        
        # Construct the output row dictionary with required fields
        output_row = {
            "customerId": row['customer_id'],  # Unique customer identifier
            "digitalLoanAccountId": row['digitalLoanAccountId'],  # Loan account ID
            "crifApplicationId": str(uuid.uuid4()),  # Random UUID for application ID
            "prediction": row.get(a, 0),  # Prediction score from specified column
            "start_time": current_time,  # Timestamp when processing starts
            "end_time": current_time,    # Timestamp when processing ends
            "modelDisplayName": modelDisplayName,  # Name of the model used
            "modelVersionId": "v1",  # Static model version
            "calcFeature": json.dumps(calc_feature, default=str),  # Features as JSON string
            "subscription_name": subscription_name,  # Subscription name
            "message_id": str(uuid.uuid4()),  # Random UUID for message ID
            "publish_time": current_time,  # Timestamp when message is published
            "attributes": "{}",  # Placeholder for additional attributes
            "trenchCategory": tc,  # Optional trench category
            "deviceOs": row['osType'],  # Device operating system
        }
        
        # Append the transformed row to the output list
        output_data.append(output_row)
    
    # Convert the list of dictionaries to a DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Return the transformed DataFrame
    return output_df


In [ ]:
# import pandas as pd
# import json
# import uuid
# from datetime import datetime
# from typing import List

# def transform_data(d1: pd.DataFrame, feature_column: List[str], a='demo_score', modelDisplayName = 'Cash_beta_trench1_Demo_backscore', tc= None, do = 'android', subscription_name = 'sil_march 25 models'):
#     # Read the input CSV file
#     df = d1.copy()
    
#     # Create the output DataFrame with the required structure
#     output_data = []
    
#     for _, row in df.iterrows():
#         # Create the calcFeature JSON with all the feature columns
#         feature_columns = feature_column
        
#         calc_feature = {}
#         for col in feature_columns:
#             if col in row and pd.notna(row[col]):
#                 # Convert Timestamp objects to string
#                 if isinstance(row[col], pd.Timestamp):
#                     calc_feature[col] = row[col].isoformat()
#                 else:
#                     calc_feature[col] = row[col]
        
       
#         # Get current timestamp
#         current_time = datetime.now().isoformat()
        
#         # Create the output row
#         output_row = {
#             "customerId": row['customer_id'],
#             "digitalLoanAccountId": row['digitalLoanAccountId'],
#             "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
#             "prediction": row.get(a, 0),
#             "start_time": current_time,
#             "end_time": current_time,
#             "modelDisplayName":modelDisplayName,
#             "modelVersionId":"v1",
#             "calcFeature": json.dumps(calc_feature, default=str),  # Use default=str to handle non-serializable objects
#             "subscription_name": subscription_name,
#             "message_id": str(uuid.uuid4()),  # Generate random UUID
#             "publish_time": current_time,
#             "attributes": "{}",  # Empty JSON object
#             "trenchCategory": tc,
#             "deviceOs": do, 
            
#         }
        
#         output_data.append(output_row)
    
#     # Create DataFrame from the output data
#     output_df = pd.DataFrame(output_data)
    
#     return output_df

# # Example usage:
# # transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# # transform_data(f'{LOCALPATH}/{transformeddata}.csv')

#### Query from risk_mart.sil_risk_ds_master_20230101_20250309_v2

##### 'Alpha - CIC-SIL-Model'

In [ ]:
sq = """select distinct
    r.customerId customer_id ,
    r.digitalLoanAccountId,
    r.cic_score,
    r.cic_Personal_Loans_granted_contracts_amt_24M,
    r.cic_days_since_last_inquiry, 
    r.cic_cnt_active_contracts,
    r.cic_vel_contract_nongranted_cnt_12on24,
    r.cic_max_amt_granted_24M, 
    r.cic_zero_non_granted_ever_flag,
    r.cic_tot_active_contracts_util,
    r.cic_vel_contract_granted_amt_12on24,
    r.cic_zero_granted_ever_flag,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from risk_mart.sil_risk_ds_master_20230101_20250309_v2 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId

where cic_score is not null;"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

In [ ]:
feature_column = ['cic_Personal_Loans_granted_contracts_amt_24M',
       'cic_days_since_last_inquiry', 'cic_cnt_active_contracts',
       'cic_vel_contract_nongranted_cnt_12on24', 'cic_max_amt_granted_24M',
       'cic_zero_non_granted_ever_flag', 'cic_tot_active_contracts_util',
       'cic_vel_contract_granted_amt_12on24', 'cic_zero_granted_ever_flag']

In [ ]:
dfd = transform_data(data, feature_column, a='cic_score', modelDisplayName='Alpha - CIC-SIL-Model') 
dfd.head()

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

##### Alpha Sil Stack Model 

In [ ]:
sq = """ 
select distinct 
r.customerId customer_id ,
r.digitalLoanAccountId,
r.alpha_stack_score,
r.beta_demo_score,
r.cic_score,
r.apps_score,
r.credo_gen_score,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where alpha_stack_score is not null
;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

In [ ]:
feature_column = ['beta_demo_score',
       'cic_score', 'apps_score',
       'credo_gen_score']

dfd = transform_data(data, feature_column, a='alpha_stack_score', modelDisplayName='Alpha - StackingModel') 
dfd.head()


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

##### Beta Sil App Score

In [ ]:
sq = """ 
select distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.apps_score,
r.app_cnt_absence_tag_30d,
r.app_cnt_absence_tag_90d ,
r.app_cnt_business_ever ,
r.app_cnt_competitors_30d ,
r.app_cnt_competitors_90d ,
r.app_cnt_education_ever ,
r.app_cnt_finance_7d ,
r.app_cnt_finance_90d ,
r.app_cnt_music_and_audio_ever ,
r.app_cnt_payday_90d ,
r.app_cnt_rated_for_3plus_ever ,
r.app_cnt_travel_and_local_ever ,
r.app_first_competitors_install_to_apply_days ,
r.app_first_payday_install_to_apply_days ,
r.app_median_time_bw_installed_mins_30d ,
r.app_vel_finance_30_over_365 ,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

In [ ]:
feature_column = ['app_cnt_rated_for_3plus_ever',
       'app_cnt_education_ever', 'app_cnt_business_ever',
       'app_cnt_music_and_audio_ever',
       'app_cnt_travel_and_local_ever', 'app_cnt_finance_7d',
       'app_cnt_competitors_30d', 'app_cnt_absence_tag_30d',
        'app_cnt_absence_tag_90d',
       'app_cnt_finance_90d', 'app_cnt_competitors_90d',
       'app_cnt_payday_90d',
       'app_median_time_bw_installed_mins_30d',
       'app_first_competitors_install_to_apply_days',
       'app_first_payday_install_to_apply_days',
       'app_vel_finance_30_over_365']

dfd = transform_data(data, feature_column, a='apps_score', modelDisplayName='Beta - AppsScoreModel') 
dfd.head()

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

##### Beta SIL Demo Score

In [ ]:
sq = """
select distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_demo_score,
r.beta_de_ln_vas_opted_flag ,
r.beta_de_ln_doc_type_rolled ,
r.beta_de_ln_marital_status ,
r.beta_de_ln_age_bin ,
r.beta_de_ln_province_bin ,
r.beta_de_ln_ref2_type ,
r.beta_de_ln_education_level ,
r.beta_de_ln_ref1_type ,
r.beta_de_ln_industry_new_bin ,
r.beta_de_ln_appln_day_of_week ,
r.beta_de_onb_name_email_match_score ,
r.beta_de_ln_employment_type_new_bin ,
r.beta_de_ln_telconame ,
r.beta_de_time_bw_onb_loan_appln_mins ,
r.beta_de_ln_source_of_funds_new_bin ,
r.beta_de_ln_brand_bin ,
r.beta_de_ln_email_primary_domain ,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where beta_demo_score is not null
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

In [ ]:
feature_column = ['beta_de_ln_vas_opted_flag',
       'beta_de_ln_doc_type_rolled', 'beta_de_ln_marital_status',
       'beta_de_ln_age_bin', 'beta_de_ln_province_bin',
       'beta_de_ln_ref2_type', 'beta_de_ln_education_level',
       'beta_de_ln_ref1_type', 'beta_de_ln_industry_new_bin',
       'beta_de_ln_appln_day_of_week',
       'beta_de_onb_name_email_match_score',
       'beta_de_ln_employment_type_new_bin', 'beta_de_ln_telconame',
       'beta_de_time_bw_onb_loan_appln_mins',
       'beta_de_ln_source_of_funds_new_bin', 'beta_de_ln_brand_bin',
       'beta_de_ln_email_primary_domain']

dfd = transform_data(data, feature_column, a='beta_demo_score', modelDisplayName='Beta - DemoScoreModel') 
dfd.head()

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

##### Beta SIL STACK Score Model

In [ ]:
sq = """ 
select  distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_stack_score,
r.apps_score,
r.credo_gen_score,
r.beta_demo_score,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where beta_stack_score is not null
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

In [ ]:
feature_column = ['apps_score', 'credo_gen_score', 'beta_demo_score']
dfd = transform_data(data, feature_column, a='beta_stack_score', modelDisplayName='Beta - StackScoreModel') 
dfd.head()

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

##### Alpha  - IncomeEstimationModel

In [ ]:
sq = """  
Select 
distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.alpha_estimated_income,
r.inc_alpha_cic_credit_avg_credit_limit,
r.inc_alpha_cic_max_active_contracts_amt,
r.inc_alpha_ln_company_name,
r.inc_alpha_ln_age,
r.inc_alpha_doc_type_rolled,
r.inc_alpha_ln_brand,
r.inc_alpha_ln_city,
r.inc_alpha_ln_cnt_dependents,
r.inc_alpha_ln_education_level,
r.inc_alpha_ln_employment_type_new,
r.inc_alpha_ln_gender,
r.inc_alpha_ln_industry_new,
r.inc_alpha_ln_loan_prod_type,
r.inc_alpha_ln_marital_status_new,
r.inc_alpha_ln_nature_of_work_new,
r.inc_alpha_ln_osversion_bin,
r.inc_alpha_ln_purpose,
r.inc_alpha_ln_source_of_funds_new,
r.inc_alpha_loan_monthly_income,
r.inc_alpha_encoded_company_name_grouped,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.alpha_estimated_income is not null
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


In [ ]:
feature_column = ['inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped']
dfd = transform_data(data, feature_column, a='alpha_estimated_income', modelDisplayName='Alpha  - IncomeEstimationModel') 
dfd.head()

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

##### Beta - IncomeEstimationModel

In [ ]:
sq = """ 
select
distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_estimated_income,
r.inc_beta_ln_loan_type,
r.inc_beta_ln_education_level,
r.inc_beta_ln_employment_type_new,
r.inc_beta_ln_industry_new,
r.inc_beta_ln_age,
r.inc_beta_ln_brand,
r.inc_beta_ln_city,
r.inc_beta_ln_purpose,
r.inc_beta_ln_osversion_bin,
r.inc_beta_ln_postal_code,
r.inc_beta_ln_gender,
r.inc_beta_ln_doc_type_rolled,
r.inc_beta_ln_cnt_dependents,
r.inc_beta_ln_source_of_funds_new,
r.inc_beta_ln_marital_status_new,
r.inc_beta_encoded_company_name_grouped,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.alpha_estimated_income is not null
;
"""

# data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
data = client.query(sq).result().to_arrow().to_pandas()
print(f"The shape of the dataframe is:\t {data.shape}")

In [ ]:
feature_column = ['inc_beta_ln_loan_type',
       'inc_beta_ln_education_level', 'inc_beta_ln_employment_type_new',
       'inc_beta_ln_industry_new', 'inc_beta_ln_age', 'inc_beta_ln_brand',
       'inc_beta_ln_city', 'inc_beta_ln_purpose', 'inc_beta_ln_osversion_bin',
       'inc_beta_ln_postal_code', 'inc_beta_ln_gender',
       'inc_beta_ln_doc_type_rolled', 'inc_beta_ln_cnt_dependents',
       'inc_beta_ln_source_of_funds_new', 'inc_beta_ln_marital_status_new',
       'inc_beta_encoded_company_name_grouped',]

dfd = transform_data(data, feature_column, a='beta_estimated_income', modelDisplayName='Beta - IncomeEstimationModel') 
dfd.head()

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

# Cash

##### Alpha Cash Stack Model

###### Trench 1

In [ ]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench1_applied_loans_backscored_20241001_20250930 r;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

In [ ]:
data.columns

In [ ]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

###### Trench 2

In [ ]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench2_applied_loans_backscored_20241001_20250930 r;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

In [ ]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

###### Trench 3

In [ ]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench3_applied_loans_backscored_20241001_20250930 r;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

In [ ]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

In [ ]:
sq ="""select modelDisplayName, count(digitalLoanAccountId) loans, count(distinct digitalLoanAccountId) cntloans 
from prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
group by 1;
"""

# dd = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

dd = client.query(sq).result().to_arrow().to_pandas()

In [ ]:
dd